In [ ]:
import os, sys, time, random
import requests as r 
import json
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np
import pandas as pd
import clickhouse_driver
from clickhouse_driver.client import Client
from tqdm.notebook import tqdm

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [ ]:
clickhouse = Client('85.193.83.20', database = 'hn_launches',
                    user = '{user}', password = '{password}')

In [131]:
headers__list = \
    [{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
      'Accept-Encoding': 'gzip, deflate, br', 
      'Accept-Language': 'en-US,en;q=0.9',},
     {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
      'Accept-Encoding': 'gzip, deflate, br', 
      'Accept-Language': 'en-US,en;q=0.9',},
     {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
      'Accept-Encoding': 'gzip, deflate, br', 
      'Accept-Language': 'en-GB,en;q=0.9',},
     {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15; rv:100.0) Gecko/20100101 Firefox/100.0',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
      'Accept-Encoding': 'gzip, deflate, br', 
      'Accept-Language': 'en-GB,en;q=0.9',},]

history = ['https://techcrunch.com', 'https://www.crunchbase.com', 'http://google.com', 'https://www.google.com/search?q=hacker+news', 'https://news.ycombinator.com', 'https://news.ycombinator.com/launches']

def create_session():
    
    global s
    if 's' in globals(): del s
    
    session = r.Session()
    session.headers.update(random.choice(headers__list))
    [session.get(i) for i in history]
    
    return session

#def rotate_ip()

In [ ]:
url = 'https://news.ycombinator.com/launches'
pages_count = 10

launches_table = pd.DataFrame(columns = ['item_id', 'title', 'url'])

for i in tqdm(range(pages_count)):
    
    time.sleep(np.random.poisson(5))
    
    s = create_session()
    
    response = s.get(url if i == 0 else url+'?next='+str(next_item_id))

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_ = 'itemlist')
        launches = table.find_all('tr', class_ = 'athing')

        next_item_id = table.find('a', class_ = 'morelink').attrs['href'].replace('launches?next=', '')

        for launch in launches:
            launches_table = launches_table.append(
                {'item_id': launch['id'], 
                 'title': launch.find('a', class_ = 'titlelink').text,
                 'url': launch.find('a', class_ = 'titlelink')['href']},
                ignore_index = True)
                
        launches_table.append(launches_table)

    else: 
        print(f'Error: {response.status_code}')
        break
        
    if next_item_id == None: break

launches_table = launches_table.astype({'item_id': 'int'})

launches_table.info()
launches_table.head()

In [ ]:
clickhouse.execute('insert into hn_launches.launches_raw values',
                   launches_table.to_dict('records'), types_check = True)

In [ ]:
item_url = 'https://news.ycombinator.com/item?id='

launches_pages = pd.DataFrame(columns = ['item_id', 'score', 'by', 'time', 'text'])
launches_comments = pd.DataFrame(columns = ['item_id', 'comment_id', 'by', 'time', 'level', 'comment'])

#attempts = 0
#while attempts < 10:

s = create_session()
for count, item_id in enumerate(tqdm(launches_table['item_id'][:60])):

    time.sleep(np.random.poisson(10))
    
    if count % 10 == 0: 
        s = create_session()
        
    #if count % 90 == 0: #select new ip
    
    s.headers.update({'referer': url+'/'})
    response = s.get(item_url+str(item_id))

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        launch_page = soup.find('table', class_ = 'fatitem')

        launches_pages = launches_pages.append(
            {'item_id': item_id,
             'score': launch_page.find('span', class_ = 'score').text.replace('points', ''),
             'by': launch_page.find('a', class_ = 'hnuser').text,
             'time': launch_page.find('span', class_ = 'age')['title'],
             'text': max([text.text for text in launch_page.find_all('td')], key = len)},
            ignore_index = True)
        
        launches_pages.append(launches_pages)
        
        comments_table = soup.find('table', class_ = 'comment-tree')

        comments = soup.find_all('tr', class_ = 'athing comtr')

        for comment in comments:
            launches_comments = launches_comments.append(
                {'item_id': item_id,
                 'comment_id': comment.attrs['id'],
                 'by': '' if any(d in comment.find('div', class_ = 'comment').text for d in ['[deleted]', '[dead]']) \
                          else comment.find('a', class_ = 'hnuser').text,
                 'time': comment.find('span', class_ = 'age').attrs['title'],
                 'level': comment.find('td', class_ = 'ind').attrs['indent'],
                 'comment': comment.find('div', class_ = 'comment').text.replace('\n', '')},
                ignore_index = True)
        
        launches_comments.append(launches_comments)

    else:
        print(f'Error: {response.status_code}')
        #attempts += 1
        #if response.status_code == 403: 
            #mark ip as banned
            #select new ip
        break

launches_pages = launches_pages.astype({'item_id': 'int', 'score': 'int'})
launches_pages['time'] = pd.to_datetime(launches_pages['time']) 

launches_comments = launches_comments.astype({'item_id': 'int', 'comment_id': 'int', 'level': 'int'})
launches_comments['time'] = pd.to_datetime(launches_comments['time']) 


display(launches_pages.head())
display(launches_comments.head())

In [ ]:
clickhouse.execute('insert into hn_launches.launches_pages_raw values',
                   launches_pages.to_dict('records'), types_check = True)

In [ ]:
clickhouse.execute('insert into hn_launches.comments_raw values', 
                   launches_comments.to_dict('records'), types_check = True)